In [ ]:
!pip install python-dotenv tavily-python langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
import operator
import os
from datetime import datetime
from typing import Annotated, TypedDict, Union
from dotenv import load_dotenv
from tavily import TavilyClient
from langchain_core.tools import tool
from langgraph.graph import END, StateGraph
from langgraph.prebuilt import ToolExecutor, ToolInvocation

# Load environment variables
load_dotenv()
TAVILY_API_KEY=""
# Initialize Tavily client
tavily = TavilyClient(TAVILY_API_KEY)

@tool
def web_search(query: str):
    """Perform a web search for the given query using Tavily."""
    try:
        # Use Tavily's search method
        search_result = tavily.search(query=query, max_results=5)

        # Extract and format the results
        return [
            {
                "title": result.get("title", "No Title"),
                "link": result.get("url", ""),
                "snippet": result.get("content", "No content")
            }
            for result in search_result.get("results", [])
        ]
    except Exception as e:
        return f"Error during Tavily search: {str(e)}"

# Prepare tools
tools = [web_search]
tool_executor = ToolExecutor(tools)

class AgentState(TypedDict):
    input: str
    intermediate_steps: Annotated[list[tuple], operator.add]
    tool_call: Union[ToolInvocation, None]

def execute_tool(state: AgentState):
    tool_call = state['tool_call']
    response = tool_executor.invoke(tool_call)
    return {"intermediate_steps": [(tool_call, response)]}

def determine_tool(state: AgentState):
    input_text = state['input']
    return {"tool_call": ToolInvocation(tool="web_search", tool_input={"query": input_text})}


workflow = StateGraph(AgentState)
workflow.add_node("select_tool", determine_tool)
workflow.add_node("execute_tool", execute_tool)
workflow.set_entry_point("select_tool")
workflow.add_conditional_edges(
    "select_tool",
    lambda state: "execute_tool" if state['tool_call'] else END,
    {
        "execute_tool": "execute_tool",
        END: END
    }
)
workflow.add_edge("execute_tool", END)



input_text_search = "Tell me about LangChain."
inputs_search = {"input": input_text_search, "intermediate_steps": []}
app_search = workflow.compile()
result_search = app_search.invoke(inputs_search)
print("\nInput:", input_text_search)
print("Result:", result_search['intermediate_steps'][0][1])


<ipython-input-11-b9e6246a5afb>:38: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)
<ipython-input-11-b9e6246a5afb>:52: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  return {"tool_call": ToolInvocation(tool="web_search", tool_input={"query": input_text})}



Input: Tell me about LangChain.
Result: [{'title': "How to Use LangChain to Build With LLMs - A Beginner's Guide", 'link': 'https://www.freecodecamp.org/news/beginners-guide-to-langchain/', 'snippet': 'LangChain is a Python library that simplifies working with large language models (LLMs) like OpenAI, Anthropic, and Google. Learn how to use LangChain to create chatbots, generate text, and stream output with examples and tutorials.'}, {'title': 'Introduction to LangChain - GeeksforGeeks', 'link': 'https://www.geeksforgeeks.org/introduction-to-langchain/', 'snippet': 'Data Structures and Algorithms\nML & Data Science\nWeb Development\nLanguages\nInterview Corner\nCS Subjects\nJobs\nPractice\nContests\nIntroduction to Langchain\nLangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). LangChain follows a general pipeline where a user asks a question to the language model where the vector representation of the question is 